In [125]:
import nltk
import json

In [3]:
nltk.download()

showing info http://nltk.github.com/nltk_data/


True

In [13]:
nltk.corpus.gutenberg.fileids()

[u'austen-emma.txt',
 u'austen-persuasion.txt',
 u'austen-sense.txt',
 u'bible-kjv.txt',
 u'blake-poems.txt',
 u'bryant-stories.txt',
 u'burgess-busterbrown.txt',
 u'carroll-alice.txt',
 u'chesterton-ball.txt',
 u'chesterton-brown.txt',
 u'chesterton-thursday.txt',
 u'edgeworth-parents.txt',
 u'melville-moby_dick.txt',
 u'milton-paradise.txt',
 u'shakespeare-caesar.txt',
 u'shakespeare-hamlet.txt',
 u'shakespeare-macbeth.txt',
 u'whitman-leaves.txt']

In [14]:
emma = nltk.corpus.gutenberg.words('austen-emma.txt')

In [8]:
c = 0
st = "hellochris"
ctx = 3
for i in range(ctx, len(st)-ctx):
    print st[i-ctx:i], st[i], st[i+1:i+ctx+1]

hel l och
ell o chr
llo c hri
loc h ris


In [9]:
def get_vals(corpus):
    for i in range(ctx, len(corpus)-ctx):
        yield (corpus[i-ctx:i], corpus[i], corpus[i+1:i+ctx+1])

In [196]:
for tp in get_vals(emma):
    right = tp[2]
    print right
    break

[u'Austen', u'1816', u']']


In [197]:
dd = dict()
c = 0
limit = 0
instances = []
attr_names = []
class_vals = set()
for tp in get_vals(emma):
    left = tp[0]
    for i in range(0, len(left)):
        left[i] = "left%i_%s" % (i, left[i])
    mid = tp[1]
    class_vals.add(mid)
    right = tp[2]
    for i in range(0, len(right)):
        right[i] = "right%i_%s" % (i, right[i])
    for k, elem in enumerate(left):
        if elem not in dd:
            dd[elem] = c
            attr_names.append(elem)
            c += 1
    if mid not in dd:
        dd[mid] = c
        #attr_names.append("mid_%s" % mid)
        c += 1
    for k, elem in enumerate(right):
        if elem not in dd:
            dd[elem] = c
            attr_names.append(elem)
            c += 1
    inst = []
    for elem in set(left+right):
        inst.append( dd[elem] )
    inst = sorted(inst)
    inst.append(mid)
    #instances.append( stringify(inst, attr_names) )
    instances.append(inst)
    limit += 1
    if limit == 100000:
        break

In [198]:
def stringify(indices, class_idx, attr_names):
    st = "{"
    for i in range(0, len(indices)):
        x = str(indices[i])
        if i == len(indices)-1:
            #x = json.dumps(attr_names[ indices[i] ])
            x = json.dumps(indices[i])
            st += str(class_idx) + " " + x
        else:
            st += ( x + " 1" )
        if i != len(indices)-1:
            st += ","
    st += "}"
    return st

In [204]:
class_idx = len(attr_names)
class_idx

34504

In [205]:
string_instances = []
n = 0
for instance in instances:
    string_instances.append( stringify(instance, class_idx, attr_names) )
    n += 1

In [140]:
[ attr_names[elem] for elem in instances[10] ]

[u'Emma', u'I', u'Woodhouse', u',', u'handsome', u'clever', u',']

In [115]:
string_instances[4]

u'{4 1,5 1,6 1,8 1,9 1,413 "VOLUME"}'

In [206]:
header = []
for attr_name in attr_names:
    header.append("@attribute %s {0,1}" % ( json.dumps(attr_name) ) )
header.append("@attribute class_attr {%s}" % ",".join( [(json.dumps(x)) for x in class_vals] ) )

In [207]:
g = open("emma.arff", "wb")
g.write("@relation emma\n")
g.write("\n".join(header) + "\n")
g.write("@data\n")
g.write("\n".join(string_instances))
g.close()

In [180]:
"mid_Jane" in attr_names

True